<a href="https://colab.research.google.com/github/chanlenium/Android-Mobile-App/blob/main/DataAnalytics/SVM_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

< SVM(Support Vector Machine) 알고리즘 >
- 두 카테고리 중 어느 하나에 속한 테이터의 집합이 주어졌을 때, SVM 알고리즘은 주어진 데이터 집합을 바탕으로 하여 새로운 데이터가 어느 카테고리에 속할지 판단하는 비확률적 이진 선형 분류 모델
- 만들어진 분류 모델은 데이터가 사상된 공간에서 경계로 표현되는데, SVM 알고리즘은 그 중 가장 큰 폭을 가진 경계를 찾음

(분석 목표) 타이타닉 데이터셋에서 탑승자의 여러 속성 데이터를 기반으로 생존여부(Survived)를 분류(예측) 함

In [ ]:
import numpy as np
import pandas as pd
import sklearn

# SVM 분류모델을 위한 패키지
from sklearn.svm import SVC
# 학습 및 테스트 데이터셋 분리를 위한 패키지
from sklearn.model_selection import train_test_split

In [ ]:
# 데이터 불러오기
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# 데이터 살펴보기 (결측치가 있는지 찾아봄)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
# 데이터 전처리 수행(Step1)
df['Age'].fillna(df['Age'].mean(), inplace=True)  # 결측치를 평균값으로 대치
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True) # 결측치를 최빈값으로 대치
df["FamilySize"] = df["SibSp"] + df["Parch"]  # 새로운 변수 생성

In [ ]:
# 데이터 전처리 수행(Step2)
# 원-핫 인코딩 : 범주형 데이터를 숫자형으로 변환. n개의 범주가 있을때, 각 범주에 해당하는 컬럼을 새로 생성하고 해당 값 유무를 0과 1로만 표시하여 계산을 단순하게 함
# Sex 컬럼의 값을 1과 0으로 원-핫 인코딩
onehot_sex = pd.get_dummies(df["Sex"])
df = pd.concat([df, onehot_sex], axis = 1)  # 원-핫 코딩한 컬럼을 기존 df에 붙임

# Embarked 컬럼의 값을 원-핫 인코딩
onehot_embarked = pd.get_dummies(df["Embarked"])
df = pd.concat([df, onehot_embarked], axis = 1)

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,female,male,C,Q,S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,1,0,1,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,1,1,0,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1,0,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,1,1,0,0,0,1
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S,0,0,1,0,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S,0,1,0,0,0,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,NaN,S,3,1,0,0,0,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C,0,0,1,1,0,0


In [ ]:
# 분석 데이터셋 준비
X = df[["Pclass", "Age", "Fare", "FamilySize", "female", "male",	"C", "Q",	"S"]] # 독립변수
y = df["Survived"]  # 종속변수

In [ ]:
# 분석 데이터셋 분할(7:3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 11)

In [ ]:
# 결과 데이터셋의 크기를 살펴봄
print(X_train.shape)  # 학습데이터(독립변수)
print(X_test.shape) # 테스트데이터(독립변수)
print(y_train.shape)  # 학습데이터(종속변수)
print(y_test.shape) # 학습데이터(종속변수)

(623, 9)
(268, 9)
(623,)
(268,)


< SVM 커널 파라미터 조정>
- SVM 알고리즘은 커널을 선택할 수 있으며, C(비용), gamma(허용 표준편차)를 이용해서 결정경계를 조절할 수 있음
- 선형 SVM은 커널을 사용하지 않고 데이터를 분류, C를 조절해서 마진의 크기를 조절하지만, 선형 분리가 주어진 차원에서 불가능할 경우 고차원으로 데이터를 옮기는 효과(커널 트릭)를 통해 결정 경계를 찾음 
- gamma가 커지면 데이터포인트별로 허용하는 표준편차가 작아지고 결정 경계도 작아지면서 구부러짐
0 파라미터 값에 따른 분석 결과를 비교하면서 최적으로 모델을 찾음

In [ ]:
# SVM객체 생성
# 1. 커널파라미터 - rbf 적용
# sv = SVC(kernel='rbf')

# 2. 커널파라미터 - linear적용, C=1, gamma=0.1
# sv = SVC(kernel='linear', C=1, gamma=0.1)

# 3. 커널파라미터 - rbf적용, C=8, gamma=0.1
sv = SVC(kernel='rbf', C=8, gamma=0.1)

# 학습 수행
sv.fit(X_train, y_train)

SVC(C=8, gamma=0.1)

In [ ]:
# 학습 완료 모델(sv)에서 테스트 데이터셋으로 예측 수행
pred = sv.predict(X_test)

In [ ]:
# 모델 성능 - 예측정확도 측정
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

0.746268656716418


In [ ]:
# 모델성능평가 - Confusion Matrix 계산
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pred)
print(mat)

[[147  29]
 [ 39  53]]


In [ ]:
# 모델성능평가 - 평가지표 계산
from sklearn.metrics import classification_report
rpt = classification_report(y_test, pred)
print(rpt)

              precision    recall  f1-score   support

           0       0.79      0.84      0.81       176
           1       0.65      0.58      0.61        92

    accuracy                           0.75       268
   macro avg       0.72      0.71      0.71       268
weighted avg       0.74      0.75      0.74       268

